<span style="color:#8B949E;">
<b>Note de lecture</b> — Notebook issu de tests itératifs (“speed-tests”).  
Le corpus utilisé ici est un <b>bilan de ces scripts de test</b> (non aligné client) uniquement pour valider la mécanique (retrieval + évaluation) et dérouler la roadmap.  
Pour le chemin complet, suivre l’ordre 01 → 10 et lire en priorité les sections Markdown.
</span>


## <span style="color:green; font-weight:700;">Phase 0</span> — <span style="color:#4DA3FF; font-weight:700;">Récupération & prise en main du corpus XML (juridique)</span>

- **Objectif** : récupérer un dump XML (textes de lois / codes / JO / jurisprudence selon source) et le **structurer “prêt pour un RAG”** : inventaire, qualité, versioning, filtrage, puis extraction/chunking. Test fait sur 1 fichier de `https://echanges.dila.gouv.fr/OPENDATA/LEGI/`
- **Constats sur les XML** : structure “riche” avec beaucoup de **liens internes** (références d’articles/sections), du **texte normatif**, et des **métadonnées** (id, num, état, dates, titres, hiérarchie, etc.) → si on indexe tout brut, on indexe du bruit.
- **Actions réalisées (POC)** :
  - sélection d’un sous-corpus (ex : *codes en vigueur*) + scripts de test progressifs ;
  - inspection manuelle de quelques XML pour repérer les balises/sections utiles vs bruit ;
  - mise en place d’un **chunking passage-level** (Script 8) : extraction du texte utile, nettoyage minimal, tracking (`chunk_index`, `doc_type`) et conservation des métadonnées.
- **Point clé “roadmap”** : une vraie mise en production d’un RAG juridique exige une **étude systématique du corpus** (inventaire des structures/balises, qualité des métadonnées, versions/dates, typologies de documents, évolutions dans le temps) afin de définir une stratégie d’extraction + chunking **robuste et maintenable**.


--- 
## 🧪 <span style="color:green; font-weight:700;">Script 1</span> — <span style="color:orange; font-weight:800;">1_BM25_simple.py</span>

### 1) Phrase d’introduction
> 🎯 **« J’ai commencé par un BM25 pur sur des documents juridiques XML afin d’identifier les limites du lexical (faux négatifs, mauvais ranking) avant d’introduire toute couche sémantique, et de garder un retrieval 100% explicable. »**

### 2) ⚙️ Fonctions
- **`extract_text_from_xml(xml_path)`**  
  Extrait tout le texte d’un XML (parcours des nœuds) pour constituer une baseline d’indexation BM25, en assumant un préprocessing minimal et donc du bruit XML.
  - **Concepts à expliciter** : baseline, bruit XML, extraction robuste, contenu indexable.

- **`tokenize(text)`**  
  Tokenise de façon volontairement simple (split) pour observer le comportement BM25 sans “masquer” les limites via une NLP plus complexe.
  - **Concepts à expliciter** : lexical pur, observabilité/debug, pas de magie.

### 3) 🔎 Analyse de fin de script
- **Ce que le script démontre** : on peut construire un retriever BM25 explicable sur corpus juridique avec une extraction minimale.
- **Limites mises en évidence** : mismatch vocabulaire utilisateur ↔ vocabulaire juridique → faux négatifs et ranking faible sur certaines requêtes.
- **Pourquoi c’est utile** : sert de socle mesurable pour justifier l’ajout progressif (benchmark + filtres + query understanding + dense + chunking + hybride) dans les scripts suivants.


--- 
## 🧪 <span style="color:green; font-weight:700;">Script 2</span> — <span style="color:orange; font-weight:800;">2_Benchmark_BM25.py</span>

### 1) Phrase d’introduction
> 🎯 **« Notre benchmark est volontairement découplé du corpus et du moteur BM25 : il consomme un corpus normalisé + un retriever afin de comparer des stratégies de retrieval sans changer la logique d’évaluation, avec métriques + VERBATIM pour diagnostiquer faux positifs et faux négatifs. »**

### 2) ⚙️ Fonctions
- **`extract_text_from_xml(xml_path)`**  
  Extrait le texte d’un fichier XML de façon robuste (approche volontairement simple) pour constituer la matière à indexer dans un POC benchmarkable.
  - **Concepts à expliciter** : extraction “baseline”, bruit XML, corpus normalisé (au sens “texte exploitable”).

- **`tokenize(text)`**  
  Tokenise simplement le texte pour alimenter BM25, afin de garder un comportement lexical observable (et donc facile à expliquer en entretien).
  - **Concepts à expliciter** : tokenisation minimaliste, retrieval lexical, observabilité.

- **`bm25_search(query, documents, bm25, top_k=10)`**  
  Exécute une recherche BM25 (scores puis tri) et retourne le top-k documents classés, base de notre évaluation.
  - **Concepts à expliciter** : BM25 (scoring), top-k (liste triée), ranking.

- **`is_relevant(document_text, relevant_keywords)`**  
  Implémente notre **oracle V1** : un document est marqué **PERTINENT** s’il contient au moins un des mots-clés attendus pour la question.
  - **Concepts à expliciter** : *oracle/ground truth du benchmark*, “pertinent” (critère binaire), faux positif / faux négatif.

- **`recall_at_k(results, relevant_keywords, k)`**  
  Calcule **Recall@k** (binaire) : 1 si au moins un pertinent est dans le top-k, sinon 0.
  - **Concepts à expliciter** : retrieval réussi vs échec, dépendance à k, signal “retrouve / ne retrouve pas”.

- **`reciprocal_rank(results, relevant_keywords)`**  
  Calcule le **MRR** : 1 / rang du premier pertinent (0 si aucun pertinent), donc mesure la rapidité à trouver “le bon”.
  - **Concepts à expliciter** : rang, “premier pertinent”, qualité de ranking.

- **`ndcg_at_k(results, relevant_keywords, k)`**  
  Calcule **nDCG@k** : mesure la qualité globale du classement (récompense les pertinents haut placés, pénalise les pertinents bas placés).
  - **Concepts à expliciter** : DCG/IDCG, pénalisation du mauvais ranking, vision “ranking global”.

- **`evaluate_benchmark(questions, documents, bm25, top_k=10)`**  
  Lance le benchmark sur toutes les questions et calcule les moyennes Recall@k / MRR / nDCG@k.
  - **Concepts à expliciter** : protocole reproductible, agrégation, comparaison avant/après.

- **`display_results_per_query(questions, documents, bm25, top_k=10)`**  
  Affiche le **VERBATIM par question** (top-k + marquage PERTINENT) pour expliquer les métriques et analyser les erreurs concrètes.
  - **Concepts à expliciter** : audit qualitatif, explicabilité, diagnostic (faux positifs / faux négatifs).

### 3) 🔎 Analyse de fin de script
- **Ce que ce script apporte (par rapport au script 1)** : une **boucle d’évaluation stable** (mêmes questions, mêmes métriques, même oracle V1) qui rend le retrieval **mesurable et comparable** (exactement ce que demande la mission : amélioration prouvée par mesures, pas au feeling).
- **Résultat baseline (tel que constaté dans le script)** : BM25 retrouve au moins un pertinent pour **2 questions sur 3** dans le top-10 (Recall@10 ≈ 0,667), mais le **premier pertinent** n’est pas toujours en tête (MRR ≈ 0,375) et le ranking global reste bruité (nDCG@10 ≈ 0,459).
- **Lecture “erreurs” grâce au VERBATIM** :
  - Les requêtes lexicalement discriminantes marchent bien (vocabulaire direct).
  - Les requêtes implicites/sémantiques génèrent des faux négatifs critiques (mismatch vocabulaire utilisateur ↔ vocabulaire juridique).
  - Une partie du bruit vient du périmètre corpus (documents hors sujet juridique immédiat), ce qui justifie ensuite filtrage, query understanding, dense, puis chunking.
- **Conclusion opérationnelle** : ce script fixe notre **point zéro chiffré** et rend explicables les échecs ; c’est le socle nécessaire pour démontrer des gains mesurés dans les scripts suivants.


---
## 🧪 <span style="color:green; font-weight:700;">Script 3</span> — <span style="color:orange; font-weight:800;">3_Benchmark_filtre-BM25.py</span>

### 1) Phrase d’introduction (extraite / resserrée)
> 🎯 **« Après la baseline BM25 brute, j’ai ajouté un filtrage métier minimal (par code) pour réduire le bruit avant toute couche sémantique, puis j’ai rejoué exactement le même benchmark (Recall@k, MRR, nDCG + VERBATIM) afin de quantifier objectivement l’impact du filtrage. »**

### 2) ⚙️ Fonctions
- **`tokenize(text)`**  
  Tokenisation volontairement simple pour observer les limites d’un retrieval lexical pur (BM25) sans complexifier l’analyse.
  - **Concepts à expliciter** : lexical, observabilité, baseline.

- **`bm25_search(query, documents, bm25, top_k=10)`**  
  Calcule les scores BM25 sur le corpus, trie par score décroissant et retourne le top-k (la liste classée utilisée par les métriques et le verbatim).
  - **Concepts à expliciter** : scoring, ranking, top-k, “moteur” BM25.

- **`is_relevant(document_text, relevant_keywords)`**  
  Implémente l’**oracle V1** : marque un document **PERTINENT** s’il contient au moins un des mots-clés attendus pour la question.
  - **Concepts à expliciter** : oracle/ground truth du benchmark, pertinent (critère binaire), faux positif / faux négatif.

- **`recall_at_k(results, relevant_keywords, k)`**  
  Calcule **Recall@k** (binaire) : 1 si au moins un pertinent est dans le top-k, sinon 0.
  - **Concepts à expliciter** : “retrouvé vs non retrouvé”, sensibilité à k.

- **`reciprocal_rank(results, relevant_keywords)`**  
  Calcule **MRR** : 1 / rang du premier pertinent (0 si aucun pertinent) pour mesurer la vitesse d’accès au premier bon résultat.
  - **Concepts à expliciter** : rang, “premier pertinent”, qualité de ranking.

- **`ndcg_at_k(results, relevant_keywords, k)`**  
  Calcule **nDCG@k** : mesure la qualité globale du classement (récompense les pertinents haut placés, pénalise les pertinents bas placés).
  - **Concepts à expliciter** : ranking global, pénalisation du mauvais ordre.

- **`evaluate_benchmark(questions, documents, bm25, top_k=10)`**  
  Exécute le benchmark sur toutes les questions et agrège les métriques moyennes (Recall@k, MRR, nDCG@k) pour comparer des variantes à protocole constant.
  - **Concepts à expliciter** : protocole reproductible, comparaison “toutes choses égales”.

- **`display_results_per_query(questions, documents, bm25, top_k=10)`**  
  Produit le **VERBATIM par question** : top-k + marquage PERTINENT, pour expliquer les métriques et diagnostiquer précisément les erreurs.
  - **Concepts à expliciter** : audit qualitatif, explicabilité, diagnostic des faux positifs / faux négatifs.

### 3) 🔎 Analyse de fin de script
- **Changement testé** : uniquement le **périmètre documentaire** (filtrage métier en amont, via `corpus_loader.py`), sans changer ni les questions ni les métriques.
- **Hypothèse** : en réduisant le bruit (documents hors périmètre), on réduit les **faux positifs** et on améliore potentiellement le **ranking** (MRR/nDCG), tout en surveillant le risque de perdre du **recall** si le filtre est trop agressif.
- **Apport clé** : l’expérience démontre une méthode “production-friendly” : *périmètre constant + benchmark constant* → on peut comparer proprement BM25, dense et hybride ensuite, sans biais expérimental.
- **Transition logique** : une fois le bruit réduit, on peut introduire le dense/embeddings avec une lecture plus nette des gains (ce que font les scripts suivants).


---
## 🧪 <span style="color:green; font-weight:700;">Script 4</span> — <span style="color:orange; font-weight:800;">4_dense_retrieval_embedding.py</span>

### 1) Phrase d’introduction
> 🎯 **« Pour mesurer l’apport réel du retrieval sémantique, j’ai gardé exactement le même corpus filtré, le même benchmark et les mêmes métriques que pour BM25, et j’ai uniquement remplacé le retriever par des embeddings : toute variation observée est donc attribuable au dense retrieval. »**

### 2) ⚙️ Fonctions
- **`is_relevant(document_text, relevant_keywords)`**  
  Implémente l’**oracle V1** : un document est marqué **PERTINENT** s’il contient au moins un des mots-clés attendus pour la question.
  - **Concepts à expliciter** : oracle/ground truth du benchmark, “pertinent” (binaire), faux positif / faux négatif.

- **`recall_at_k(results, relevant_keywords, k)`**  
  Calcule **Recall@k** (binaire) : 1 si au moins un pertinent apparaît dans le top-k, sinon 0.
  - **Concepts à expliciter** : succès retrieval vs échec, dépendance à k.

- **`reciprocal_rank(results, relevant_keywords)`**  
  Calcule **MRR** : 1 / rang du premier pertinent (0 si aucun), donc mesure la qualité de ranking “au premier bon résultat”.
  - **Concepts à expliciter** : rang, “premier pertinent”, vitesse d’accès au bon extrait.

- **`ndcg_at_k(results, relevant_keywords, k)`**  
  Calcule **nDCG@k** : mesure la qualité globale du classement (récompense les pertinents en haut de liste).
  - **Concepts à expliciter** : ranking global, pénalisation des pertinents mal classés.

- **`dense_search(query, documents, document_embeddings, embedding_model, top_k=10)`**  
  Réalise une recherche sémantique : encode la requête en embedding, calcule la **similarité cosinus** avec les embeddings des documents, trie et retourne le top-k.
  - **Concepts à expliciter** : embeddings (vecteurs), encodeur, similarité cosinus, retrieval sémantique, top-k.

- **`evaluate_dense_benchmark(questions, documents, document_embeddings, embedding_model, k=10)`**  
  Exécute le benchmark dense sur toutes les questions et agrège les métriques (Recall@k, MRR, nDCG@k) pour comparer proprement avec BM25.
  - **Concepts à expliciter** : protocole constant, comparaison “un seul changement”, amélioration prouvée par mesures.

- **`display_dense_results_per_query(questions, documents, document_embeddings, embedding_model, k=10)`**  
  Affiche le **VERBATIM par question** : top-k + similarité + drapeau **PERTINENT/NON PERTINENT**, pour interpréter les métriques et analyser les gains/échecs du sémantique.
  - **Concepts à expliciter** : audit qualitatif, explicabilité, diagnostic (mismatch lexical vs sémantique).

### 3) 🔎 Analyse de fin de script
- **Changement testé** : remplacement de BM25 par un **dense retriever** (embeddings + similarité cosinus), sans changer le corpus filtré, les questions, ni les métriques.
- **Ce que ce script permet de démontrer en entretien** :
  - on sait isoler l’effet d’un composant (ici : le retriever) → approche **mesurable, stable, améliorable** ;
  - le dense retrieval est particulièrement pertinent quand la formulation utilisateur est **sémantique / paraphrasée** (synonymes, notions implicites) là où le lexical peut échouer.
- **Ce que le VERBATIM permet de trancher** :
  - si le gain vient d’un meilleur **recall** (on retrouve enfin un pertinent),
  - ou d’un meilleur **ranking** (le pertinent existait déjà mais remonte dans le top-k),
  - ou au contraire si le dense introduit du **hors-sujet sémantiquement proche** (faux positifs).
- **Transition logique** : si le dense améliore certaines requêtes mais reste bruité, la suite naturelle est (1) mieux structurer la requête (query understanding), puis (2) améliorer la granularité du corpus (chunking) et enfin (3) hybrider BM25+dense (RRF) pour cumuler signaux lexical + sémantique.



---
## 🧪 <span style="color:green; font-weight:700;">Script 5</span> — <span style="color:orange; font-weight:800;">5_bm25_with_query_understanding.py</span>

### 1) Phrase d’introduction
> 🎯 **« Pour coller à l’attendu “question utilisateur → bons extraits”, j’ai introduit une brique déterministe de *query understanding* (dictionnaire métier) afin d’enrichir la requête avant retrieval, puis j’ai comparé à protocole constant BM25 “requête brute” vs BM25 “requête enrichie” (métriques avant/après). »**

### 2) ⚙️ Fonctions
- **`tokenize(text)`**  
  Tokenise le texte en mots (regex) pour alimenter BM25 de façon stable et explicable.
  - **Concepts à expliciter** : retrieval lexical, tokens, observabilité.

- **`bm25_search(query, k=10)`**  
  Calcule les scores BM25 de la requête sur tout le corpus, trie les couples (document, score), et retourne le top-k.
  - **Concepts à expliciter** : scoring, ranking, top-k, baseline explicable.

- **`evaluate_bm25(use_query_understanding=False)`**  
  Exécute le benchmark sur toutes les questions en optionnant l’enrichissement de requête :  
  - si `False` → requête brute,  
  - si `True` → requête enrichie via `process_user_query(...)` (dictionnaire métier),  
  puis calcule les moyennes **Recall@10 / MRR / nDCG@10**.
  - **Concepts à expliciter** :
    - **Query understanding (déterministe)** : transformation contrôlée de la requête, audit-able, versionnable.
    - **`enriched_query`** : texte final réellement envoyé au retriever (important pour prouver ce qui est testé).
    - **Oracle V1** (implicite via `relevant_keywords`) : règle qui définit “PERTINENT” pour mesurer Recall/MRR/nDCG.

### 3) 🔎 Analyse de fin de script
- **But de l’expérience** : vérifier si un enrichissement métier “sans LLM” améliore déjà un retriever lexical (BM25), et surtout établir une méthode “avant/après” mesurable.
- **Lecture attendue des résultats** :
  - Si l’enrichissement ajoute des termes juridiques absents de la requête, BM25 peut gagner (meilleur recall ou meilleur rang du premier pertinent).
  - Si la requête brute contenait déjà le vocabulaire principal, l’effet sur BM25 peut être faible (normal : BM25 reste lexical).
- **Pourquoi c’est utile pour la mission** :
  - on démontre une brique *stable et améliorable* (dictionnaire versionnable),
  - on instrumente l’impact avec des métriques (pas au feeling),
  - et on prépare la suite : ce même enrichissement devient souvent plus payant sur **dense** et **hybride** (scripts suivants), où la reformulation sémantique compte davantage.



---
## 🧪 <span style="color:green; font-weight:700;">Script 6</span> — <span style="color:orange; font-weight:800;">6_bm25_filtered_with_query_understanding.py</span>

### 1) Phrase d’introduction (extraite / resserrée)
> 🎯 **« Après BM25 + query understanding, j’ajoute un filtrage métier explicite (“Code du travail”) sur le corpus avant indexation, puis je rejoue le même benchmark pour mesurer si la réduction de bruit + l’enrichissement de requête améliorent Recall@10 / MRR / nDCG@10. »**

### 2) ⚙️ Fonctions
- **`tokenize(text)`**  
  Découpe le texte en tokens (mots) en minuscules via regex afin d’alimenter BM25 de façon stable et reproductible.
  - **Concepts à expliciter** : tokenisation, lexical, normalisation (lowercase), reproductibilité.

- **`bm25_search(query, k=10)`**  
  Calcule les scores BM25 de la requête sur les **documents filtrés** (`filtered_documents`), trie par score décroissant et retourne le top-k.
  - **Concepts à expliciter** : scoring BM25, ranking, top-k, périmètre filtré, réduction du bruit.

- **`evaluate()`**  
  Exécute le benchmark sur toutes les questions : enrichit chaque requête via `process_user_query(..., dictionary)`, interroge BM25, puis agrège **Recall@10 / MRR / nDCG@10**.
  - **Concepts à expliciter** :
    - **Query understanding** : enrichissement déterministe via dictionnaire (`enriched_query` = requête réellement envoyée au retriever).
    - **Oracle V1** : la pertinence est évaluée via `relevant_keywords` (mots-clés attendus).
    - **Mesure “avant/après”** : protocole constant, seul le filtre + enrichissement changent.

### 2bis) 🧰 Variables / blocs importants
- **`dictionary = load_juridical_dictionary(...)`** : charge le référentiel métier (intentions utilisateur → concepts juridiques → cibles).
- **`filtered_documents = [...] if "code du travail" in doc["text"].lower()`** : filtrage périmètre “métier” (réduit le bruit inter-code).
- **`bm25 = BM25Okapi(tokenized_docs)`** : index BM25 construit uniquement sur le périmètre filtré.

### 3) 🔎 Analyse de fin de script
- **Hypothèse testée** : en réduisant le bruit (filtrage “Code du travail”) *et* en enrichissant la requête (dictionnaire métier), on augmente :
  - soit le **recall** (on retrouve enfin un pertinent dans le top-10),
  - soit la **qualité de ranking** (le pertinent remonte → MRR/nDCG).
- **Ce que ce script démontre méthodologiquement** : une approche “production-friendly” où la performance dépend autant de la **qualité du périmètre (filtre)** et de la **normalisation de requête** que du modèle.
- **Limite assumée** : le filtre textuel `"code du travail"` reste une heuristique (utile en POC) ; l’étape suivante logique est de filtrer via métadonnées/identifiants plus robustes et/ou de passer au passage-level (chunking) pour mieux aligner question ↔ extrait.


---
## 🧪 <span style="color:green; font-weight:700;">Script 7</span> — <span style="color:orange; font-weight:800;">7_dense_with_query_understanding.py</span>

### 1) Phrase d’introduction (extraite / resserrée)
> 🎯 **« J’ai évalué un retrieval sémantique (embeddings) combiné à une requête enrichie par dictionnaire métier, à protocole constant (mêmes questions, mêmes métriques), pour mesurer l’impact réel sur la chaîne “question utilisateur → bons extraits”. »**

### 2) ⚙️ Fonctions
- **`dense_search(query, k=10)`**  
  Encode la requête en embedding, calcule la similarité cosinus avec les embeddings des documents, trie par score décroissant et renvoie le top-k.
  - **Concepts à expliciter** :
    - **Embeddings** : représentation vectorielle du texte (sémantique).
    - **`normalize_embeddings=True`** : normalisation des vecteurs pour rendre la cosinus cohérente.
    - **Cosine similarity** : score de proximité sémantique utilisé pour classer les documents.
    - **Top-k** : liste ordonnée des meilleurs candidats “bons extraits” (avant génération).

- **`evaluate()`**  
  Pour chaque question du benchmark, enrichit la requête via `process_user_query(..., dictionary)` (on utilise `enriched_query`), lance `dense_search`, puis calcule et moyenne **Recall@10**, **MRR**, **nDCG@10** via l’oracle V1 (`relevant_keywords`).
  - **Concepts à expliciter** :
    - **Query understanding (déterministe)** : enrichissement versionnable/auditable basé sur un dictionnaire métier.
    - **`enriched_query`** : requête réellement envoyée au retriever (ce qui est mesuré).
    - **Oracle V1** : règle de pertinence binaire “PERTINENT” basée sur des mots-clés attendus (`relevant_keywords`).
    - **Recall@10 / MRR / nDCG@10** : mesure du retrieval (retrouve / rang du 1er pertinent / qualité globale du ranking).

### 2bis) 🧰 Blocs/variables importants
- **`dictionary = load_juridical_dictionary("juridical_dictionary.yml")`** : charge le référentiel métier (intentions utilisateur → concepts juridiques → cibles).
- **`model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")`** : encodeur d’embeddings multilingue (FR OK).
- **`doc_embeddings = model.encode(doc_texts, normalize_embeddings=True, show_progress_bar=True)`** : pré-calcul des embeddings documents (performance + stabilité), pour éviter de ré-encoder à chaque requête.

### 3) 🔎 Analyse de fin de script
- **Protocole strict** : mêmes questions, mêmes métriques, seul le retriever change → les gains observés sont attribuables au dense retrieval + requête enrichie.
- **Résultats (tels que synthétisés en fin de script)** :
  - BM25 (requête brute / enrichie / filtrée+enrichie) : **Recall@10 = 0,33**, **MRR = 0,33**, **nDCG@10 = 0,33**.
  - Dense + requête enrichie (ce script) : **Recall@10 = 0,67**, **MRR ≈ 0,38**, **nDCG@10 ≈ 0,40**.
- **Interprétation** :
  - Le gain principal est un **meilleur recall** (on retrouve un pertinent sur 2 questions sur 3 au lieu de 1 sur 3).
  - Le dense “amplifie” une requête mieux posée : la couche métier (query understanding) sert à rapprocher le langage utilisateur du langage juridique.
- **Implication “production RAG”** :
  - On peut prouver une amélioration sans “feeling” (métriques avant/après).
  - Le retrieval dense est une brique utile, mais doit rester pilotée par (1) compréhension métier explicite et (2) audit/benchmark, avant d’aller vers chunking et hybride.


---
## 🧪 <span style="color:green; font-weight:700;">Script 8</span> — <span style="color:orange; font-weight:800;">8_corpus_chunker_xml.py</span>

### 1) Phrase d’introduction
> 🎯 **« J’ai transformé les XML juridiques en un corpus “passage-level” (JSONL) : des chunks propres, traçables et filtrables, afin d’améliorer la précision du retrieval et de rendre l’évaluation/audit (pertinence, citations, erreurs) réellement exploitables pour un RAG. »**

### 2) ⚙️ Fonctions
- **`normalize_text(text)`**  
  Normalise un texte (trim + espaces multiples) pour stabiliser l’indexation et l’affichage.
  - **Concepts** : normalisation, stabilité du corpus.

- **`strip_technical_header(text)`**  
  Retire des en-têtes techniques parasites (ID/URL/stats concaténés) en préfixe pour éviter de polluer les chunks.
  - **Concepts** : bruit technique, nettoyage conservateur.

- **`safe_findtext(node, xpath, default="")`**  
  Récupère un champ XML (findtext) de façon sûre (valeur par défaut) pour éviter les crashs sur XML incomplets.
  - **Concepts** : robustesse, XML hétérogène.

- **`iter_text_nodes(node)`**  
  Itère sur le texte utile des nœuds XML (texte + tails) pour construire un contenu “lisible” à partir de structures XML variées.
  - **Concepts** : extraction robuste, texte “utile”.

- **`strip_namespaces(xml_string)`**  
  Supprime les namespaces XML pour simplifier les XPath et rendre l’extraction plus stable.
  - **Concepts** : namespaces, XPath, robustesse parsing.

- **`infer_doc_type_from_path(xml_path)`**  
  Déduit un `doc_type` à partir du chemin/fichier (ex: article, section, etc.) pour permettre des filtres métier ensuite.
  - **Concepts** : typologie documentaire, filtrage par métadonnées.

- **`_looks_like_abbrev_before_dot(s, idx)`**  
  Heuristique pour détecter des abréviations avant un point afin d’éviter de “casser” une phrase au mauvais endroit.
  - **Concepts** : segmentation fiable, abréviations juridiques.

- **`split_into_sentences(text)`**  
  Segmente un texte en phrases (heuristique) pour préparer un chunking “phrase-aware”.
  - **Concepts** : phrase-aware, segmentation.

- **`split_long_text_phrase_aware(text, max_chars=1200)`**  
  Découpe un texte long en sous-parties proches de `max_chars` en privilégiant les frontières de phrases.
  - **Concepts** : `max_chars`, chunking contrôlé, qualité de lecture/citation.

- **`extract_article_meta(root, xml_path)`**  
  Extrait des métadonnées d’article (ex: numéro, titre, identifiants) pour tracer et évaluer la pertinence plus proprement.
  - **Concepts** : `meta.num` (numéro d’article), traçabilité, évaluation “article-aware”.

- **`extract_links(root, max_links=20)`**  
  Collecte un échantillon de liens (audit) et les stocke en métadonnées pour inspection/filtrage éventuel.
  - **Concepts** : audit de corpus, liens comme bruit/indice.

- **`_walk_collect_paragraphs(node, paragraphs)`**  
  Parcourt l’arbre XML et accumule des paragraphes textuels utiles en évitant les zones peu pertinentes.
  - **Concepts** : extraction structurée, sélection de contenu.

- **`extract_paragraphs(xml_path)`**  
  Extrait une liste de paragraphes “propres” depuis un XML (cœur du passage-level : on part de `<p>` et assimilés).
  - **Concepts** : passage-level, granularité, réduction de bruit.

- **`chunk_paragraphs(paragraphs, max_chars=1200, overlap_paragraphs=1)`**  
  Regroupe des paragraphes en chunks passage-level avec une taille cible et un léger overlap pour préserver le contexte.
  - **Concepts** : `max_chars`, `overlap_paragraphs`, contexte vs précision.

- **`stable_chunk_id(doc_id, chunk_index, chunk_text)`**  
  Génère un identifiant de chunk stable (reproductible) qui reflète le contenu réellement indexé.
  - **Concepts** : stabilité, déduplication, traçabilité.

- **`iter_xml_files(data_root)`**  
  Itère sur les XML d’un répertoire racine (traversal) pour batcher le traitement.
  - **Concepts** : pipeline corpus, scalabilité.

- **`build_chunk_corpus_for_file(xml_path, cfg)`**  
  Construit les enregistrements chunkés (dicts) pour un fichier : extraction → nettoyage → meta → chunking.
  - **Concepts** : transformation “end-to-end” par document.

- **`build_chunk_corpus(data_root, cfg)`**  
  Applique le chunking sur tout le corpus et agrège tous les chunks dans une liste d’enregistrements JSONL.
  - **Concepts** : dataset passage-level, reproductibilité.

- **`save_jsonl(records, out_path)`**  
  Sauvegarde les chunks au format JSONL (1 chunk = 1 ligne) pour indexation et benchmarks ultérieurs.
  - **Concepts** : JSONL, ingestion simple, compatibilité tooling.

- **`print_quick_stats(records)`**  
  Affiche des stats rapides (nb docs, nb chunks, longueurs min/moy/max) pour vérifier l’effet du chunking.
  - **Concepts** : contrôle qualité, dérive de corpus.

- **`default_run_config()`**  
  Fournit une config par défaut (seuils de chunking, overlap, filtres de qualité, options liens) pour runs reproductibles.
  - **Concepts** : config explicite, reproductibilité.

- **`parse_args(argv=None)`**  
  Parse les arguments CLI (data_root, out_jsonl, max_chars, overlap, min_text_len, liens, limit_files).
  - **Concepts** : CLI, expérimentation contrôlée.

- **`run_with_config(cfg)`**  
  Lance un run complet avec une config : build corpus chunké → sauvegarde → stats/audit.
  - **Concepts** : pipeline exécutable, run déterministe.

- **`main(argv=None)`**  
  Point d’entrée : mode “config par défaut” (sans args) ou mode CLI explicite.
  - **Concepts** : ergonomie dev, exécution sécurisée.

### 3) 🔎 Analyse de fin de script
- **Objectif atteint** : convertir des XML juridiques en un dataset **passage-level JSONL** permettant un retrieval plus précis (top-k = extraits directement citables) et une évaluation plus robuste (pertinence au niveau passage, pas document entier).
- **Résultats de run (ordre de grandeur)** : **~4 091 documents** sources → **~13 180 chunks** ; extraction de liens en métadonnées (audit) avec **~26,8 liens/chunk** en moyenne (utile pour décider d’un filtrage ultérieur).
- **Contrôles qualitatifs réalisés** : inspection de cas informatifs (chunk le plus long, chunk très court, présence de headers techniques, liens, cohérence de segmentation) pour valider que le chunking produit du texte “retrievable”.
- **Pourquoi c’est clé pour la mission** : un RAG “production” ne se joue pas seulement sur le modèle ; il faut un corpus **structuré, traçable, filtrable** pour (1) améliorer la pertinence, (2) faciliter l’annotation/guide d’évaluation, (3) rendre l’amélioration chiffrable et explicable.


---
## 🧪 <span style="color:green; font-weight:700;">Script 9</span> — <span style="color:orange; font-weight:800;">9_benchmark_bm25_on_jsonl_chunks.py</span>

### 1) Phrase d’introduction
> 🎯 **« Je benchmarke BM25 sur un corpus *passage-level* (chunks JSONL issus du script 8) afin de mesurer, de façon chiffrée et audit-able (métriques + verbatim), l’impact du chunking et d’améliorations “petites mais explicables” (query understanding et expansion métier minimale) sur la chaîne “question utilisateur → bons extraits”. »**

### 2) ⚙️ Fonctions
- **`tokenize(text)`**  
  Tokenisation simple (minuscules + mots alphanumériques) pour un BM25 lisible et “debuggable”.
  - **Concepts à expliciter** : BM25 = lexical, tokenisation baseline, reproductibilité.

- **`expand_query_minimal_legal(query)`**  
  Ajoute une **expansion métier minimale** (heuristique) à la requête pour réduire les échecs BM25 dus au mismatch lexical, sans complexifier (objectif : petit / explicable).
  - **Concepts à expliciter** : expansion contrôlée, mismatch lexical, amélioration “cheap but effective”.

- **`get_dedupe_value(doc, dedupe_key)`**  
  Extrait une valeur stable (ex: `meta.num`, `meta.id`, `doc_id`) servant à **dédupliquer** un ranking.
  - **Concepts à expliciter** : déduplication = éviter 10 variantes du même article, `dedupe_key` = clé de regroupement.

- **`dedupe_ranked_results(ranked_results, dedupe_key)`**  
  Déduplique une liste triée `(doc, score)` en conservant le **premier** document pour chaque valeur de `dedupe_key`.
  - **Concepts à expliciter** : ranking trié, déduplication “first wins”, lisibilité du top-k.

- **`bm25_search(query, bm25, documents, top_k)`**  
  Retourne le top-k `(doc, score)` selon BM25 (scoring + tri décroissant).
  - **Concepts à expliciter** : score BM25, ranking, top-k, baseline explicable.

- **`is_relevant_by_keywords(text, relevant_keywords)`**  
  Implémente l’**oracle V1** : un document/chunk est affiché **PERTINENT** s’il contient au moins un mot-clé attendu (`relevant_keywords`).
  - **Concepts à expliciter** : oracle/ground truth du benchmark, “pertinent” = critère binaire, faux positif / faux négatif.

- **`evaluate_bm25(queries, documents, bm25, top_k, use_query_understanding, use_minimal_expansion, dedupe_key)`**  
  Exécute le benchmark et calcule **Recall@k, MRR, nDCG@k** en testant 4 modes : requête brute/enrichie × avec/sans expansion minimale, avec option de déduplication.
  - **Concepts à expliciter** : protocole constant, comparaison “un seul changement”, métriques de retrieval (avant génération).

- **`display_results_per_query(queries, documents, bm25, top_k, use_query_understanding, use_minimal_expansion, dedupe_key)`**  
  Affiche le **verbatim par question** : top-k + scores + drapeau **PERTINENT** (oracle), avec déduplication optionnelle.
  - **Concepts à expliciter** : audit qualitatif, interprétation des métriques, diagnostic des erreurs.

- **`parse_args(argv=None)`**  
  Parse les arguments CLI et utilise `parse_known_args()` pour rester robuste en environnement type Spyder/IPython.
  - **Concepts à expliciter** : robustesse tooling, exécution reproductible.

- **`parse_csv_list(s)`**  
  Parse une liste CSV (ex: `"article,section_ta"`) en nettoyant les espaces.
  - **Concepts à expliciter** : ergonomie CLI, paramètres contrôlés.

- **`load_jsonl_documents_robust(path, **kwargs)`**  
  Charge les chunks JSONL via `corpus_loader_jsonl.load_documents` en gérant plusieurs signatures possibles (robustesse si le loader évolue).
  - **Concepts à expliciter** : compatibilité, stabilité du pipeline, dette technique maîtrisée.

- **`main()`**  
  Orchestration : charge corpus JSONL chunké, prépare BM25, lance l’évaluation + verbatim selon les options CLI.
  - **Concepts à expliciter** : pipeline “end-to-end” benchmarkable.

### 3) 🔎 Analyse de fin de script
- **Protocole** : benchmark BM25 sur **chunks JSONL** (passage-level) avec :
  - métriques **Recall@10 / MRR / nDCG@10**,
  - **verbatim top-k** pour expliquer les chiffres,
  - options : requête enrichie (query understanding) et/ou **expansion métier minimale**, et déduplication du top-k (`dedupe_key`).

- **Résultats (tels que synthétisés dans le script)** :
  - **BM25 chunks — requête brute** : Recall@10 **0,667** ; MRR **0,444** ; nDCG@10 **0,500**
  - **BM25 chunks — requête enrichie** : Recall@10 **0,667** ; MRR **0,444** ; nDCG@10 **0,500**
  - **BM25 chunks — brute + expansion minimale** : Recall@10 **0,667** ; MRR **0,667** ; nDCG@10 **0,667**
  - **BM25 chunks — enrichie + expansion minimale** : Recall@10 **0,667** ; MRR **0,667** ; nDCG@10 **0,667**

- **Interprétation (quantitatif)** :
  - Le **Recall@10 reste stable** (2 questions sur 3 “hit” dans le top-10) : le chunking aide mais BM25 reste limité sur certaines requêtes.
  - L’**expansion métier minimale** améliore fortement le **ranking** :
    - **MRR** augmente (le premier pertinent remonte souvent au rang 1),
    - **nDCG@10** augmente (top-10 mieux ordonné, pertinents plus haut).

- **Interprétation (qualitatif via verbatim)** :
  - Sur les requêtes “lexicalisées”, BM25 sur chunks fonctionne bien.
  - L’échec persistant sur une requête plus “sémantique” illustre une limite attendue : BM25 remonte des passages proches lexicalement (structure, termes génériques) sans tomber sur l’extrait normatif précis.

- **Ce qu’il faut valoriser en entretien (aligné mission)** :
  1) Passage-level + benchmark = retrieval **mesurable, stable, améliorable** (on prouve les gains par métriques + audit).  
  2) Une amélioration **simple, explicable et versionnable** (expansion minimale) peut donner un gain net de ranking sans changer de modèle.  
  3) Les échecs restants deviennent des signaux exploitables : ils justifient l’étape suivante (dense/hybride ou filtrage plus fin des passages “plan/structure”) plutôt que du “tuning au feeling”.


---
## 🧪 <span style="color:green; font-weight:700;">Script 10</span> — <span style="color:orange; font-weight:800;">10_benchmark_hybride_rrf_bm25_dense_chunks.py</span>

### 1) Phrase d’introduction
> 🎯 **« Notre objectif est de rendre la chaîne “question utilisateur → bons extraits” mesurable et améliorable : ce script benchmarke, sur un corpus chunké JSONL filtrable, trois retrievers (BM25, Dense, Hybride via RRF) avec un oracle V2 “article-aware” et un verbatim aligné sur l’évaluation, pour expliquer chaque succès/échec sans ambiguïté. »**

### 2) ⚙️ Fonctions
- **`load_chunks_jsonl(jsonl_path, limit)`**  
  Charge le corpus chunké JSONL (1 ligne = 1 chunk dict) avec option de limite.
  - **Concepts** : JSONL passage-level, chunk = unité de retrieval/citation.

- **`chunk_matches_title(chunk, needle)`**  
  Teste si un chunk matche un critère de titre (via `meta.titre`, sinon fallback `doc_id`).
  - **Concepts** : filtre “title_contains”, robustesse si meta partielle.

- **`filter_chunks(chunks, allowed_doc_types, title_contains, collection_contains)`**  
  Filtre le corpus par `doc_type`, par sous-chaîne dans titre/doc_id, et par identifiant de collection dans `doc_id`.
  - **Concepts** : périmètre contrôlé, réduction de bruit, reproductibilité des tests.

- **`normalize_for_bm25(text)`**  
  Normalise le texte (minuscule/espaces) pour stabiliser BM25.
  - **Concepts** : normalisation, stabilité du scoring lexical.

- **`tokenize(text)`**  
  Tokenisation BM25 robuste (liste de tokens) adaptée à un corpus juridique bruité.
  - **Concepts** : lexical pur, observabilité/debug.

- **`build_bm25_index(documents)`**  
  Construit l’index BM25 sur `documents[i]["text"]`.
  - **Concepts** : baseline explicable, indexation.

- **`bm25_search(query, documents, bm25, top_k)`**  
  Retourne la liste triée `(doc_index, score)` BM25 dans le top-k.
  - **Concepts** : ranking, top-k, score BM25.

- **`_cache_key()`**  
  Fabrique une clé de cache **stable** (incluant les paramètres de filtrage) pour éviter un mauvais ré-emploi d’embeddings.
  - **Concepts** : anti-désalignement, cache “cohérent avec le dataset”.

- **`build_dense_embeddings(documents, model_name, cache_dir, cache_tag)`**  
  Construit ou recharge la matrice d’embeddings des chunks (cache disque).
  - **Concepts** : performance, reproductibilité, embeddings Sentence-Transformers.

- **`dense_search_all(query, model, doc_emb, top_k)`**  
  Dense search sur **tout** le corpus (cosine similarity) et renvoie le top-k indices.
  - **Concepts** : similarité cosinus, retrieval sémantique global.

- **`dense_rerank_subset(query, model, doc_emb, subset_indices, top_k)`**  
  Rerank dense sur un **sous-ensemble** (utile en hybride : reranker seulement une shortlist).
  - **Concepts** : rerank, shortlist, efficacité en production.

- **`hybrid_rrf_search(query, documents, bm25, model, doc_emb, top_k, shortlist_k, rrf_k)`**  
  Hybride BM25 + Dense via **RRF (Reciprocal Rank Fusion)** : fusionne deux rankings en pondérant par le rang.
  - **Concepts** : RRF, fusion de signaux lexical+sémantique, `shortlist_k` (taille shortlist), `rrf_k` (constante RRF).

- **`minimal_business_expansion(query)`**  
  Expansion “métier” ultra-minimale (heuristique) indépendante du dictionnaire, pour aider le retrieval sans boîte noire.
  - **Concepts** : amélioration simple/explicable, réduction du mismatch lexical.

- **`_safe_lower(x)`**  
  Lowercase sûr (gère `None`) pour éviter des erreurs lors des accès meta/texte.
  - **Concepts** : robustesse.

- **`_q_get(q, key, default)`**, **`q_id(q)`**, **`q_question(q)`**, **`q_num_prefixes(q)`**, **`q_keywords_fallback(q)`**  
  Accès robuste aux champs d’une query (ID, texte, préfixes attendus, fallback mots-clés).
  - **Concepts** : stabilité du benchmark, compatibilité V1/V2.

- **`is_relevant_v2(chunk, q)`**  
  **Oracle V2** : un chunk est **PERTINENT** si `meta.num` est présent et matche un des `relevant_num_prefixes` attendus (sinon fallback possible).
  - **Concepts** : oracle/ground truth, “article-aware”, pertinence binaire explicable.

- **`recall_at_k_v2(ranked_indices, documents, q, k)`**  
  Recall@k (V2) : 1 si au moins un chunk pertinent est dans le top-k, sinon 0.
  - **Concepts** : “retrouve / ne retrouve pas”.

- **`reciprocal_rank_v2(ranked_indices, documents, q)`**  
  MRR (V2) : 1/rang du premier pertinent (0 si aucun).
  - **Concepts** : “premier pertinent”, ranking.

- **`ndcg_at_k_v2(ranked_indices, documents, q, k)`**  
  nDCG@k (V2) en pertinence binaire : qualité globale du classement.
  - **Concepts** : ranking global, pertinents plus haut = meilleur score.

- **`get_query_variant(q, dictionary, use_query_understanding, use_min_expansion)`**  
  Construit la requête effective envoyée au retriever : brute / enrichie via dictionnaire / + expansion minimale.
  - **Concepts** : “requête réellement testée”, auditabilité, alignement expérimental.

- **`evaluate_retriever(name, queries, documents, retriever_fn, top_k, metrics_k)`**  
  Calcule Recall@k, MRR, nDCG@k sur toutes les queries pour un retriever donné (BM25, Dense, Hybride).
  - **Concepts** : protocole constant, comparaison mesurée (pas au feeling).

- **`show_verbatim(queries, documents, retriever_fn, top_k, label)`**  
  Affiche le top-k par requête avec marquage **PERTINENT** (oracle V2) : audit qualitatif et diagnostic d’erreurs.
  - **Concepts** : verbatim aligné sur l’évaluation, explicabilité.

- **`parse_args()`**  
  Parse les options CLI (filtres `doc_types`, `collection_contains`, `title_contains`, tailles `top_k`, `bm25_shortlist`, etc.).
  - **Concepts** : runs reproductibles, paramètres contrôlés.

- **`main()`**  
  Orchestration complète : chargement + filtrage chunks → BM25 → embeddings (cache) → benchmarks BM25/Dense/Hybride + verbatim.

### 3) 🔎 Analyse de fin de script
- **Ce que ce script verrouille (qualité d’expérimentation)** :
  - **Oracle V2 cohérent** (pertinence par `meta.num` + préfixes attendus) → mesure plus “juridique” que du simple mot-clé.
  - **Verbatim aligné** : ce qui est évalué est exactement ce qui est affiché (pas d’artefact d’affichage).
  - **Cache embeddings anti-désalignement** : les embeddings correspondent bien au corpus filtré utilisé au benchmark.

- **Ce que le dernier run montre (lecture produit, pas “bug”)** :
  - Sur le périmètre filtré (exemple : ~386 chunks “article” d’une collection), une requête peut réussir nettement (ex : définition du licenciement économique) tandis que d’autres restent en échec (ex : “sans préavis”, “contester”) → ce n’est plus un problème de métriques, mais un problème de **signal retrieval / formulation / périmètre**.

- **Interprétation utile pour la mission (“question → bons extraits”)** :
  - Les échecs restants sont désormais **diagnosticables** : soit le signal lexical+sémantique “licenciement” attire vers un mauvais chapitre, soit les formulations attendues sont dispersées/hors périmètre, soit l’enrichissement actuel n’est pas assez discriminant vis-à-vis de l’oracle V2.
  - 1 seul dataset xml étant testé sur des centaines, il se peut aussi que notre corpus de test soit trop limité ;-)
  - Prochaine marche “production-grade” (sans relancer des tests ici) : **routing/contraintes structurelles** (par typologie/chapitre), enrichissement métier plus ciblé, et/ou reranking plus fort sur shortlist (dense rerank) — toujours avec métriques + verbatim.
